# Thingiverse Data Processing
w/ Alex's Dataset<p>

In [125]:
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import numpy as np
import pickle
from textblob import TextBlob
import re

In [126]:
aa="aqqq sd. sd"
qqq='sd'
re.search(r'\b'+qqq+r'\b', aa)

<re.Match object; span=(5, 7), match='sd'>

In [127]:
pd.set_option('display.max_columns', False)

In [128]:
client = MongoClient('mongodb://localhost:27017')

In [129]:
db=client.alexthing
thing_info=db.sample

In [130]:
thing_info.count_documents({})

305

In [131]:
f2=thing_info.find({}).limit(1)
for document in f2:
    print('################')
    pprint(document)

################
{'_id': ObjectId('5e6aed3b5fd388f326c80ea7'),
 'author': 'Chuk59',
 'author_url': 'https://www.thingiverse.com/Chuk59',
 'author_url_img': 'https://cdn.thingiverse.com/renders/39/23/3f/46/a3/00ff9d84e04f5c5e5620397410051379_thumb_small.jpg',
 'banner_challenge_winner': False,
 'banner_education': False,
 'banner_featured': False,
 'banner_verified': False,
 'category': '/fashion/costume',
 'collection_number': '29',
 'comments': [{'author': 'Chuk59',
               'author_img': 'https://cdn.thingiverse.com/renders/39/23/3f/46/a3/00ff9d84e04f5c5e5620397410051379_thumb_small.jpg',
               'author_url': 'https://www.thingiverse.com/Chuk59',
               'comment_id': '1292261',
               'date': 'Apr 16, 2017 4:58 pm',
               'depth': '1',
               'parent': None,
               'text': '<a rel="nofollow" '
                       'href="https://i.imgur.com/KLg9lnU.jpg" '
                       '>https://i.imgur.com/KLg9lnU.jpg</a>  '},
       

In [132]:
dfs = pd.DataFrame(list(thing_info.find({},{'id':1,'details_parts':1,'title':1,'category':1})))

In [133]:
dfs = dfs.reindex(columns = list(dfs.columns)+['Is3DModel','FileExt','FileCt','FirstImg','ImgCt','DesignTools','Materials','SortedWords'])

In [134]:
dfs.head()

,_id,title,id,category,Is3DModel,FileExt,FileCt,FirstImg,ImgCt,DesignTools,Materials,SortedWords
0,5e6aed3b5fd388f326c80ea7,Deadpool katana support,1897299,/fashion/costume,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5e6aed3b5fd388f326c80ea8,My cookie cutter,1897303,/household/kitchen-and-dining,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5e6aed3b5fd388f326c80ea9,Sunny Motor V2 Parts,1897307,/hobby/diy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5e6aed3b5fd388f326c80eaa,My Customized Lego Sandcastle Mold,1897327,/toys-and-games,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5e6aed3b5fd388f326c80eab,Arch iPad Stand,189730,/gadgets/tablet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
df=dfs.copy()

In [136]:
df.head()

,_id,title,id,category,Is3DModel,FileExt,FileCt,FirstImg,ImgCt,DesignTools,Materials,SortedWords
0,5e6aed3b5fd388f326c80ea7,Deadpool katana support,1897299,/fashion/costume,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5e6aed3b5fd388f326c80ea8,My cookie cutter,1897303,/household/kitchen-and-dining,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5e6aed3b5fd388f326c80ea9,Sunny Motor V2 Parts,1897307,/hobby/diy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5e6aed3b5fd388f326c80eaa,My Customized Lego Sandcastle Mold,1897327,/toys-and-games,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5e6aed3b5fd388f326c80eab,Arch iPad Stand,189730,/gadgets/tablet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
excluded_words=["makerbot","piece","crafty","phillip","times","inch","inches","ends","purpose","design","designs","revision","action","figures","lots","prototypes","chance","drop","mission","set","vector","file","pieces","token","instructions","day","http","https","www","noun","none",'i','you','he','she','it',"test","zip","lots","sizes"]

def update_word_count(s, weight, nn_dict, pos_tag):
    s=s.lower()
    blob = TextBlob(s)
    for w in [n for n,t in blob.tags if t in pos_tag]:
        if w in excluded_words or not w.isalpha():
            continue
        if w in nn_dict:
            nn_dict[w]+=weight
        else:
            nn_dict[w]=weight

In [138]:
def remove_html(s):
    re_rule = re.compile('<.*?>')
    return re.sub(re_rule, '', s)

In [139]:
design_tools=["123D Catch","123D Design","3D Tin","Blender","Thingiverse Customizer","Illustrator","Inkscape","Inventor","Mathematica","Maya","MeshLab","MeshMixer","OpenSCAD","Photoshop","PrintShop","Rhino","Sculptris","Sketchup","Solidworks","Tinkercad","TopMod","ZBrush","FreeCAD","Fusion 360","Microsoft 3D Builder","3D Builder","CATIA","Autodesk 3ds Max","3ds Max","Orchard"]
def check_design_tools(s):
    s=s.lower()
    matched_tools=[]
    for tool in design_tools:
        if tool.lower() in s:
            matched_tools.append(tool)
    return matched_tools

In [144]:
materials=['ABS','PLA','wood']
def check_material(s):
    s=s.lower()
    matched_materials=[]
    for m in materials:
        if re.search(r'\b'+m.lower()+r'\b', s):
            matched_materials.append(m)
    return matched_materials

In [145]:
fd=thing_info.find({},{'tags':1,'files':1,'description':1,'images':1,'title':1})

i=0
name_w=5
summary_w=1
instruction_w=0
tag_w=3
pos_tag=['NN','NNS']
three_d_extensions=['stl','3ds','obj','fbx','dae','stp','prt','ply','ipt']

design_tools=["123D Catch","123D Design","3D Tin","Blender","Thingiverse Customizer","Illustrator","Inkscape","Inventor","Mathematica","Maya","MeshLab","MeshMixer","OpenSCAD","Photoshop","PrintShop","Rhino","Sculptris","Sketchup","Solidworks","Tinkercad","TopMod","ZBrush","FreeCAD","Fusion 360","Microsoft 3D Builder","3D Builder","CATIA","Autodesk 3ds Max","3ds Max","Orchard"]

for document in fd:
    #File
    if len(document['files'])>0:
        df.loc[i,'FileCt']=len(document['files'])
        fname=document['files'][0]['filename'].lower()
        file_extension=fname[fname.rfind('.')+1:]
        df.loc[i,'FileExt']=fname[fname.rfind('.')+1:]
        df.loc[i,'Is3DModel']=file_extension in three_d_extensions
    
    #Images
    if len(document['images'])>0:
        df.loc[i,'ImgCt']=len(document['images'])
        df.loc[i,'FirstImg']='=IMAGE(\"'+document['images'][0]+'\")'
        
    #Sorted Words
    nn_dict={}
    tools_used=[]
    materials_used=[]
    update_word_count(document['title'],name_w,nn_dict,pos_tag)
    tag_s=' '.join(t[0] for t in document['tags'])
    tools_used+=check_design_tools(tag_s)
    materials_used+=check_material(tag_s)
    update_word_count(tag_s, tag_w, nn_dict,pos_tag)
    for part in document['description']:
        if 'text' in part:
            t=remove_html(part['text'])
            tools_used+=check_design_tools(t)
            materials_used+=check_material(t)
            if part['header']=='Summary':
                update_word_count(t,summary_w,nn_dict,pos_tag)
            elif part['header']=='Instructions':
                update_word_count(t,instruction_w,nn_dict,pos_tag)
    
    w_list=sorted(nn_dict, key=nn_dict.get)[::-1]
    df.loc[i,'SortedWords']=', '.join(w_list)
    df.loc[i,'DesignTools']=', '.join(tools_used)
    df.loc[i,'Materials']=', '.join(materials_used)
    
    i+=1

In [146]:
df.head()

,_id,title,id,category,Is3DModel,FileExt,FileCt,FirstImg,ImgCt,DesignTools,Materials,SortedWords
0,5e6aed3b5fd388f326c80ea7,Deadpool katana support,1897299,/fashion/costume,True,stl,1.0,"=IMAGE(""https://cdn.thingiverse.com/renders/14...",2.0,,,"deadpool, support, katana, scabbard"
1,5e6aed3b5fd388f326c80ea8,My cookie cutter,1897303,/household/kitchen-and-dining,True,stl,1.0,"=IMAGE(""https://cdn.thingiverse.com/renders/2f...",1.0,,,"cutter, cookie, customizer, version, wall, hei..."
2,5e6aed3b5fd388f326c80ea9,Sunny Motor V2 Parts,1897307,/hobby/diy,True,stl,2.0,"=IMAGE(""https://cdn.thingiverse.com/renders/08...",2.0,,,"parts, motor, paraglider, paramotor, video, li..."
3,5e6aed3b5fd388f326c80eaa,My Customized Lego Sandcastle Mold,1897327,/toys-and-games,True,stl,1.0,"=IMAGE(""https://cdn.thingiverse.com/renders/25...",1.0,,,"mold, sandcastle, customizer, version, size, l..."
4,5e6aed3b5fd388f326c80eab,Arch iPad Stand,189730,/gadgets/tablet,True,stl,1.0,"=IMAGE(""https://cdn.thingiverse.com/renders/46...",2.0,,,"arch, stand, ipad, tablet, degree"


In [147]:
df.to_csv('thing_sample.csv')